In [ ]:
import pandas as pd
import os
from scipy.stats import shapiro
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

# 월별 CD금리 파일 불러오기
cd_df = pd.read_csv('./csv/월별_CD금리.csv')


# 펀드 파일 전처리
# 경로설정
path = './excel/주식형펀드/'

# 빈 리스트 생성
file_list = []

# 경로 내 모든 xls파일을 file_list에 저장
for filename in os.listdir(path):
    if filename.endswith('.xls'):
        file_list.append(filename)

# 파일 정렬
file_list.sort()
# 빈 데이터 프레임 생성
full_df = pd.DataFrame()

# 리스트에 저장된 파일 하나씩 불러오기
for file in file_list:
    # print(file)
    fund_df = pd.read_excel(path+file)

    # 필요한 컬럼만 남긴다
    # 기준일자, 기준가격, 전일대비\n등락, 벤치마크지수
    fund_df = fund_df[["기준일자", "기준가격", "벤치마크지수"]]

    # 컬럼명 변경
    fund_df.rename(columns={"벤치마크지수" : "KOSPI"}, inplace=True)
    
    # 불필요 행 제거
    fund_df = fund_df.loc[1:]
    
    # 기준일자 오름차순으로 변경
    fund_df.sort_values("기준일자", ascending=True, inplace=True)    
    
    # 인덱스 재설정
    fund_df.reset_index(drop=True, inplace=True)    
    
    # 기준월 컬럼 생성
    fund_df["기준일자"] = fund_df["기준일자"].str.replace("/", "")
    fund_df["기준월"] = fund_df["기준일자"].astype(str).str[:6]    
    
    # 일별수익률 컬럼 생성 및 계산
    fund_df["일별수익률"] = fund_df["기준가격"].pct_change()    
    
    # KOSPI 일별수익률 컬럼 생성 및 계산
    fund_df["KOSPI 일별수익률"] = fund_df["KOSPI"].pct_change()    
    
    # 결측치 제거
    fund_df.dropna(inplace=True)
    fund_df.reset_index(drop=True, inplace=True)
    
    # 수익률 표준편차
    monthly_std = fund_df.groupby(["기준월"])["일별수익률"].std(ddof=0)
    monthly_std = pd.DataFrame(monthly_std)
    
    # KOSPI 수익률 표준편차
    kospi_monthly_std = fund_df.groupby(["기준월"])["KOSPI 일별수익률"].std(ddof=0)
    kospi_monthly_std = pd.DataFrame(kospi_monthly_std)
    
    # 기준일자별 마지막 날짜
    monthly_last_day = fund_df.groupby(["기준월"])["기준일자"].max()
    monthly_last_day = pd.DataFrame(monthly_last_day)    
        
    fund_df.drop(["일별수익률", "KOSPI 일별수익률"], axis=1, inplace=True)    

    # 데이터 프레임 merge    
    df = pd.merge(monthly_last_day, fund_df, how="left", on="기준일자")   
        
    df = pd.merge(df, monthly_std, how="left", on="기준월")    
        
    df = pd.merge(df, kospi_monthly_std, how="left", on="기준월")    
        
    df.rename(columns={"일별수익률" : "월별 표준편차",
                    "KOSPI 일별수익률" : "KOSPI 월별 표준편차"}, inplace=True)    
        
    # 월별수익률 컬럼 생성
    df["월별수익률"] = df["기준가격"].pct_change()

    # KOSPI 월별수익률 컬럼 생성
    df["KOSPI 월별수익률"] = df["KOSPI"].pct_change()

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df = df[["기준일자", "기준가격", "KOSPI", "기준월", "월별수익률", "월별 표준편차", "KOSPI 월별수익률", "KOSPI 월별 표준편차"]]

    # CD금리 전처리
    # df 복사
    fund_cd_df = df.copy()

    # CD금리 컬럼 추가
    fund_cd_df["CD금리"] = cd_df["원자료"]

    fund_cd_df["CD금리"] = fund_cd_df["CD금리"]/100
    fund_cd_df.dropna(inplace=True)

    # 초과수익률 계산
    excess_return = fund_cd_df["월별수익률"]-fund_cd_df["CD금리"]
    return_std = fund_cd_df["월별 표준편차"]

    # 샤프지수 데이터 프레임 생성 및 컬럼 추가
    sharpe_df = pd.DataFrame()
    sharpe_df["기준월"] = fund_cd_df["기준월"]
    sharpe_df["초과수익률"] = excess_return
    sharpe_df["월별 표준편차"] = return_std
    sharpe_df

    # 샤프지수 계산    
    sharpe_df["샤프지수"] = ""
    sharpe_df["샤프지수"] = sharpe_df["초과수익률"].apply(lambda x : x/sharpe_df["월별 표준편차"] if x >= 0 else x*sharpe_df["월별 표준편차"])    


In [ ]:
# csv 파일로 저장
full_df.to_csv('./csv/유가증권/2021_투자자별_거래실적.csv', encoding='CP949')